# 動態網頁爬蟲 - 使用 Selenium
* 了解 Selenium 用於動態網頁爬蟲的原理
* 能夠使用 Selenium 撰寫動態網頁爬蟲

## 作業目標
* 根據範例使用 selenuim 取出 高雄市小港區 2020 年 空氣品質 資料，並用一個適合的資料結構儲存
* Hint: 將所有資料存在一個變數當中

# Beautiful Soup 4.4.0 文档
https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0/

In [1]:
from selenium import webdriver
ddriver = webdriver.Chrome()

In [2]:
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup


In [3]:
ddriver.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")


In [11]:
#測站 對應值
selectSite = Select(ddriver.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('58')
#年度 對應值
selectYear = Select(ddriver.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2020')
#selectYear.select_by_value('2019')

#查詢 點擊
ddriver.find_element_by_id('ctl05_btnQuery').click()


In [12]:
data=ddriver.page_source

In [13]:
soup= BeautifulSoup(data,'lxml')
table = soup.find('table', class_='TABLE_G')
table

<table align="Center" border="1" cellpadding="3" cellspacing="0" class="TABLE_G" id="ctl05_gv" rules="all" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">
<tbody><tr style="color:Black;font-weight:normal;">
<th scope="col">監測項目</th><th scope="col">單位</th><th scope="col">監測日期</th><th scope="col">監測值</th><th scope="col">標註</th>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="2" style="white-space:nowrap;" valign="top">SO2</td><td class="no-alt" rowspan="2" valign="top">ppb</td><td>2020/01</td><td>3</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2020/02</td><td>3.10</td><td> </td>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="2" style="white-space:nowrap;" valign="top">CO</td><td class="no-alt" rowspan="2" valign="top">ppm</td><td>2020/01</td><td>0.58</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2020/02</td><td>0.47</td><td> </td>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="2" s

In [14]:
table.findAll('td')

[<td class="no-alt" rowspan="2" style="white-space:nowrap;" valign="top">SO2</td>,
 <td class="no-alt" rowspan="2" valign="top">ppb</td>,
 <td>2020/01</td>,
 <td>3</td>,
 <td> </td>,
 <td>2020/02</td>,
 <td>3.10</td>,
 <td> </td>,
 <td class="no-alt" rowspan="2" style="white-space:nowrap;" valign="top">CO</td>,
 <td class="no-alt" rowspan="2" valign="top">ppm</td>,
 <td>2020/01</td>,
 <td>0.58</td>,
 <td> </td>,
 <td>2020/02</td>,
 <td>0.47</td>,
 <td> </td>,
 <td class="no-alt" rowspan="2" style="white-space:nowrap;" valign="top">O3</td>,
 <td class="no-alt" rowspan="2" valign="top">ppb</td>,
 <td>2020/01</td>,
 <td>28.40</td>,
 <td> </td>,
 <td>2020/02</td>,
 <td>30.30</td>,
 <td> </td>,
 <td class="no-alt" rowspan="2" style="white-space:nowrap;" valign="top">PM10</td>,
 <td class="no-alt" rowspan="2" valign="top">μg/m3</td>,
 <td>2020/01</td>,
 <td>55</td>,
 <td> </td>,
 <td>2020/02</td>,
 <td>53</td>,
 <td> </td>,
 <td class="no-alt" rowspan="2" style="white-space:nowrap;" valign="

In [15]:
import re
air_data={}
for cat in table.findAll('td')[:]:
    if "class" in cat.attrs and "style" in cat.attrs :
        name=cat.text 
        air_data[name] = {}
    else:
        if re.compile("\d{4}\/\d{2}").match(cat.text):
            #print(cat.text)
            date = cat.text
            air_data[name][date] = {}
        elif re.compile("\d+").match(cat.text):
            #print(cat.text)
            value = cat.text
            air_data[name][date] = value
            
print(air_data)        

{'THC': {'2020/02': '2.12', '2020/01': '2.18'}, 'NOx': {'2020/02': '25.91', '2020/01': '29.72'}, 'PM10': {'2020/02': '53', '2020/01': '55'}, 'NO': {'2020/02': '5.45', '2020/01': '6.01'}, 'CO': {'2020/02': '0.47', '2020/01': '0.58'}, 'NMHC': {'2020/02': '0.18', '2020/01': '0.20'}, 'O3': {'2020/02': '30.30', '2020/01': '28.40'}, 'CH4': {'2020/02': '1.94', '2020/01': '1.98'}, 'NO2': {'2020/02': '20.46', '2020/01': '23.71'}, 'SO2': {'2020/02': '3.10', '2020/01': '3'}}


In [10]:
import pandas as pd
pd.DataFrame(air_data)

,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,SO2,THC
2019/01,2,0.67,0.27,7.95,30.06,38.01,24.70,78,4.10,2.27
2019/02,1.94,0.48,0.17,4.76,20.86,25.62,29.40,57,3.30,2.11
2019/03,1.94,0.50,0.18,5.41,22.43,27.84,29.20,52,3.90,2.12
2019/04,1.88,0.39,0.14,4.16,17.68,21.83,30.40,42,3.40,2.02
2019/05,1.86,0.38,0.16,4.58,17.98,22.55,28.60,35,3.30,2.02
2019/06,1.79,0.24,0.13,5.08,12.08,17.15,16.30,20,4.10,1.92
2019/07,1.80,0.28,0.14,5.16,12.54,17.71,15.50,24,4,1.93
2019/08,1.82,0.33,0.15,5.17,14.49,19.66,19.70,27,4.60,1.97
2019/09,1.86,0.32,0.13,3.63,15.03,18.66,35.70,35,3.50,1.98
2019/10,1.92,0.36,0.13,3.50,15.99,19.50,44,50,3.20,2.05


In [17]:
pd.DataFrame(air_data)

,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,SO2,THC
2020/01,1.98,0.58,0.20,6.01,23.71,29.72,28.40,55,3,2.18
2020/02,1.94,0.47,0.18,5.45,20.46,25.91,30.30,53,3.10,2.12


In [18]:
res= pd.DataFrame(air_data)
res.to_csv('air_test.csv',index=False)
